<a href="https://colab.research.google.com/github/LRLeite/Data-Analytics/blob/main/Explorando%20Dados%20com%20SQL/dvd_rental.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 📀 **Resumo do Dataset DVD Rental (PostgreSQL)**

O dataset **DVD Rental** (PostgreSQL) simula uma locadora de DVDs, ideal para explorar conceitos de banco de dados relacionais e praticar consultas complexas. Composto por 15 tabelas, como `film`, `actor`, `customer`, e `rental`, ele oferece um ambiente realista para projetos SQL e análise de dados.

---
**Fonte dos dados:** [dvd-rental](https://www.postgresqltutorial.com/postgresql-getting-started/postgresql-sample-database/)



# **Bibliotecas**

In [21]:
#Instalação de bibliotecas necessárias
#!pip install google-cloud-secret-manager google-auth
#!pip install sqlalchemy

In [2]:
#Importação de módulos
from google.colab import auth
from google.cloud import secretmanager
import pandas as pd
from sqlalchemy import create_engine

# **Conexão com o banco de dados SQL**

In [3]:
#Autenticação Google Cloud para acessar chaves do Secret Manager
auth.authenticate_user()
print("Autenticação Google Cloud concluída!")

Autenticação Google Cloud concluída!


In [4]:
#Carregamento de credenciais do Secret Manager
PROJECT_ID = "inner-topic-464213-p6"
client = secretmanager.SecretManagerServiceClient()

try:
    name_host = f"projects/{PROJECT_ID}/secrets/supabase-db-host/versions/latest"
    response_host = client.access_secret_version(name=name_host)
    DB_HOST = response_host.payload.data.decode("UTF-8")

    name_name = f"projects/{PROJECT_ID}/secrets/supabase-db-name/versions/latest"
    response_name = client.access_secret_version(name=name_name)
    DB_NAME = response_name.payload.data.decode("UTF-8")

    name_user = f"projects/{PROJECT_ID}/secrets/supabase-db-user/versions/latest"
    response_user = client.access_secret_version(name=name_user)
    DB_USER = response_user.payload.data.decode("UTF-8")

    name_password = f"projects/{PROJECT_ID}/secrets/supabase-db-password/versions/latest"
    response_password = client.access_secret_version(name=name_password)
    DB_PASSWORD = response_password.payload.data.decode("UTF-8")

    name_port = f"projects/{PROJECT_ID}/secrets/supabase-db-port/versions/latest"
    response_port = client.access_secret_version(name=name_port)
    DB_PORT = response_port.payload.data.decode("UTF-8")

    print("Credenciais do Secret Manager carregadas com sucesso!")

except Exception as e:
    print(f"Erro ao acessar segredos do Secret Manager: {e}")
    print("Verifique se o PROJECT_ID, nomes dos secrets e permissões IAM estão corretos.")

Credenciais do Secret Manager carregadas com sucesso!


In [5]:
#Criação da engine SQLAlchemy
engine = None

try:
    db_uri = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
    engine = create_engine(db_uri)
    print("\nEngine do SQLAlchemy criada com sucesso usando credenciais do Secret Manager!")

except Exception as e:
    print(f"Erro ao criar a engine do SQLAlchemy: {e}")


Engine do SQLAlchemy criada com sucesso usando credenciais do Secret Manager!


# **Statement fundamentals**

In [6]:
#Selecionar o nome, sobremnome e e-mail dos clientes
query_1 = """
SELECT first_name, last_name, email FROM customer;
"""

df_1 = pd.read_sql_query(query_1, engine)
display(df_1)

,first_name,last_name,email
0,Jared,Ely,jared.ely@sakilacustomer.org
1,Mary,Smith,mary.smith@sakilacustomer.org
2,Patricia,Johnson,patricia.johnson@sakilacustomer.org
3,Linda,Williams,linda.williams@sakilacustomer.org
4,Barbara,Jones,barbara.jones@sakilacustomer.org
...,...,...,...
594,Terrence,Gunderson,terrence.gunderson@sakilacustomer.org
595,Enrique,Forsythe,enrique.forsythe@sakilacustomer.org
596,Freddie,Duggan,freddie.duggan@sakilacustomer.org
597,Wade,Delvalle,wade.delvalle@sakilacustomer.org


In [7]:
#Selecionar os diferentes tipos de avaliações que os filmes podem apresentar
query_2 = """
SELECT DISTINCT rating FROM film;
"""

df_2 = pd.read_sql_query(query_2, engine)
print(df_2)

  rating
0     PG
1  PG-13
2  NC-17
3      R
4      G


In [8]:
#Selecionar o nome, sobrenome e e-mail de Lois Butler
query_3 = """
SELECT first_name, last_name, email
FROM customer
WHERE first_name = 'Lois' AND last_name = 'Butler';
"""

df_3 = pd.read_sql_query(query_3, engine)
display(df_3)

,first_name,last_name,email
0,Lois,Butler,lois.butler@sakilacustomer.org


In [9]:
#Selecionar a descrição do filme "Outlaw Hanky"
query_4 = """
SELECT description
FROM film
WHERE title = 'Outlaw Hanky';
"""

df_4 = pd.read_sql_query(query_4, engine)
pd.set_option('display.max_colwidth', None)
display(df_4)

,description
0,A Thoughtful Story of a Astronaut And a Composer who must Conquer a Dog in The Sahara Desert


# **Manipulação de datas**

https://www.postgresql.org/docs/current/functions-datetime.html

In [10]:
#Verificar os dias em que houveram pagamento no 1° trimestre
query_6 = """
SELECT
        DISTINCT(CAST(EXTRACT(month FROM payment_date) AS INTEGER)) AS payment_month,
        TO_CHAR(payment_date, 'MONTH') AS payment_monthName,
  TO_CHAR(payment_date, 'DD-mon-YYYY') AS formatted_date
FROM payment
WHERE EXTRACT(QUARTER FROM payment_date) = 1
ORDER BY payment_month
"""

df_6 = pd.read_sql_query(query_6, engine)
print(df_6)

    payment_month payment_monthname formatted_date
0               2         FEBRUARY     14-feb-2007
1               2         FEBRUARY     15-feb-2007
2               2         FEBRUARY     16-feb-2007
3               2         FEBRUARY     17-feb-2007
4               2         FEBRUARY     18-feb-2007
5               2         FEBRUARY     19-feb-2007
6               2         FEBRUARY     20-feb-2007
7               2         FEBRUARY     21-feb-2007
8               3         MARCH        01-mar-2007
9               3         MARCH        02-mar-2007
10              3         MARCH        16-mar-2007
11              3         MARCH        17-mar-2007
12              3         MARCH        18-mar-2007
13              3         MARCH        19-mar-2007
14              3         MARCH        20-mar-2007
15              3         MARCH        21-mar-2007
16              3         MARCH        22-mar-2007
17              3         MARCH        23-mar-2007


# **Funções matemáticas**

In [11]:
#Calcular valores agregados da tabela payment
query_7 = """
SELECT
    SUM(amount) AS total_amount,
    AVG(amount) AS average_amount,
    MIN(amount) AS min_amount,
    MAX(amount) AS max_amount
FROM payment;
"""

df_7 = pd.read_sql_query(query_7, engine)
print(df_7)

   total_amount  average_amount  min_amount  max_amount
0      61312.04        4.200606         0.0       11.99


In [12]:
#Calcular custo percentual dos filmes
query_8 = """
SELECT
    title,
    ROUND(rental_rate / replacement_cost, 2)*100 AS custo_percent
FROM film;
"""

df_8 = pd.read_sql_query(query_8, engine)
print(df_8)

                 title  custo_percent
0      Chamber Italian           33.0
1     Grosse Wonderful           25.0
2      Airport Pollock           31.0
3    Bright Encounters           38.0
4     Academy Dinosaur            5.0
..                 ...            ...
995     Young Language           10.0
996         Youth Kick            7.0
997       Zhivago Core            9.0
998  Zoolander Fiction           10.0
999          Zorro Ark           26.0

[1000 rows x 2 columns]


# **GROUP BY, HAVING, ORDER BY e LIMIT**




In [13]:
#Selecionar os 10 clientes com maior gasto total
query_9 = """
SELECT customer_id, SUM(amount) AS total_spent
FROM payment
GROUP BY customer_id
ORDER BY total_spent DESC
LIMIT 10;
"""

df_9 = pd.read_sql_query(query_9, engine)
print(df_9)

   customer_id  total_spent
0          148       211.55
1          526       208.58
2          178       194.61
3          137       191.62
4          144       189.60
5          459       183.63
6          181       167.67
7          410       167.62
8          236       166.61
9          403       162.67


In [14]:
#Verificar qual funcionário lida com a maioria dos pagamentos
query_10 = """
SELECT staff_id, COUNT(*) AS payment_count
FROM payment
GROUP BY staff_id
ORDER BY payment_count DESC
LIMIT 1;
"""

df_10 = pd.read_sql_query(query_10, engine)
print(df_10)

   staff_id  payment_count
0         2           7304


In [15]:
#Verificar a me´dia do custo de reposição para os filmes com avaliação 'G', 'PG' e 'R'
query_11 = """
SELECT rating, ROUND(AVG(replacement_cost), 2) AS avg_replacement_cost
FROM film
WHERE rating IN ('G', 'PG', 'R')
GROUP BY rating;
"""

df_11 = pd.read_sql_query(query_11, engine)
print(df_11)

  rating  avg_replacement_cost
0     PG                 18.96
1      R                 20.23
2      G                 20.12


In [16]:
#Verificar os clientes que tiveram mais de 39 transações
query_12 = """
SELECT customer_id, COUNT(*) AS transaction_count
FROM payment
GROUP BY customer_id
HAVING COUNT(*) > 39
ORDER BY COUNT(*) DESC;
"""

df_12 = pd.read_sql_query(query_12, engine)
print(df_12)

   customer_id  transaction_count
0          148                 45
1          526                 42
2          144                 40


In [17]:
#Selecionar os clientes que gastaram mais de $100 com o funcionário id 2
query_13 = """
SELECT customer_id, SUM(amount) AS total_spent
FROM payment
WHERE staff_id = 2
GROUP BY customer_id
HAVING SUM(amount) > 100
ORDER BY SUM(amount) DESC;
"""

df_13 = pd.read_sql_query(query_13, engine)
print(df_13)

   customer_id  total_spent
0          187       110.81
1          148       110.78
2          211       108.77
3          522       102.80
4          526       101.78


# **INNER JOIN**

In [18]:
#Selecionar os e-mails dos clientes que moram na Califórnia
query_14 = """
SELECT district, email FROM customer
INNER JOIN address ON customer.address_id = address.address_id
WHERE district = 'California';
"""

df_14 = pd.read_sql_query(query_14, engine)
display(df_14)

,district,email
0,California,patricia.johnson@sakilacustomer.org
1,California,betty.white@sakilacustomer.org
2,California,alice.stewart@sakilacustomer.org
3,California,rosa.reynolds@sakilacustomer.org
4,California,renee.lane@sakilacustomer.org
5,California,kristin.johnston@sakilacustomer.org
6,California,cassandra.walters@sakilacustomer.org
7,California,jacob.lance@sakilacustomer.org
8,California,rene.mcalister@sakilacustomer.org


In [19]:
#Encontrar os filmes em que Lisa Monroe participou
query_15 = """
SELECT first_name || ' ' || last_name AS name, title FROM actor
INNER JOIN film_actor ON actor.actor_id = film_actor.actor_id
INNER JOIN film ON film_actor.film_id = film.film_id
WHERE(first_name || ' ' || last_name) = 'Lisa Monroe'
"""

df_15 = pd.read_sql_query(query_15, engine)
print(df_15)

           name               title
0   Lisa Monroe   Anything Savannah
1   Lisa Monroe        Arabia Dogma
2   Lisa Monroe  Butterfly Chocolat
3   Lisa Monroe         Chitty Lock
4   Lisa Monroe       Club Graffiti
5   Lisa Monroe       Coast Rainbow
6   Lisa Monroe         Crow Grease
7   Lisa Monroe       Crusade Honey
8   Lisa Monroe    Effect Gladiator
9   Lisa Monroe   Fiction Christmas
10  Lisa Monroe       Hanky October
11  Lisa Monroe       Jericho Mulan
12  Lisa Monroe    Lesson Cleopatra
13  Lisa Monroe        Lover Truman
14  Lisa Monroe       Mod Secretary
15  Lisa Monroe         Quills Bull
16  Lisa Monroe        River Outlaw
17  Lisa Monroe      Roots Remember
18  Lisa Monroe        Sassy Packer
19  Lisa Monroe   Vacation Boondock
20  Lisa Monroe         Wild Apollo
21  Lisa Monroe           Won Dares
22  Lisa Monroe           Zorro Ark


**<h1>Encerrar conexão com o banco de dados</h1>**

In [20]:
#Disposição da engine SQLAlchemy ao final
print("\n--- Fechando Conexão ---")
if engine:
    engine.dispose()
    print("Engine SQLAlchemy disposed.")
print("Notebook finalizado.")


--- Fechando Conexão ---
Engine SQLAlchemy disposed.
Notebook finalizado.
